In [1]:
from __future__ import print_function
import pickle
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import file, client, tools
from apiclient import errors
import base64
from datetime import datetime
import re

import pandas as pd

In [2]:
def FindNumbersBraket(input_string):
    
    try:
        numbers_list = re.findall(r"\[\d-\d+\]", input_string)
        
        numbers_list = [item.replace('[', '').replace(']', '') for item in numbers_list]
        
    except:
        numbers_list = []
        
    return numbers_list

In [3]:
def DateConv(date_str):
    '''
    Input: 'Y-m-d H:M:S'
    '''
    
    ## datetime object
    date_object = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

    ## timestamp
    ts = datetime.timestamp(date_object) 
    
    
    return ts, date_object

In [4]:
def GetAttachments(service, user_id, msg_id):
    """Get and store attachment from Message with given id.

    :param service: Authorized Gmail API service instance.
    :param user_id: User's email address. The special value "me" can be used to indicate the authenticated user.
    :param msg_id: ID of Message containing attachment.
    """
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()

        for part in message['payload']['parts']:
            if part['filename']:
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body']['attachmentId']
                    att = service.users().messages().attachments().get(userId=user_id, messageId=msg_id,id=att_id).execute()
                    data = att['data']
                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                path = part['filename']

                with open(path, 'wb') as f:
                    f.write(file_data)

    except:
        print('An error occurred')
        pass

In [5]:
def GetMessage(service, user_id, msg_id):
  """Get a Message with given ID.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    msg_id: The ID of the Message required.

  Returns:
    A Message.
  """
  try:
    message = service.users().messages().get(userId=user_id, id=msg_id).execute()

    print ('Message snippet: %s' % message['snippet'])

    return message
  except:
    print('An error occurred: %s' % error)


In [6]:
def ListMessagesMatchingQuery(service, user_id, query=''):
  """List all Messages of the user's mailbox matching the query.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    query: String used to filter messages returned.
    Eg.- 'from:user@some_domain.com' for Messages from a particular sender.

  Returns:
    List of Messages that match the criteria of the query. Note that the
    returned list contains Message IDs, you must use get with the
    appropriate ID to get the details of a Message.
  """
  try:
    response = service.users().messages().list(userId=user_id,
                                               q=query).execute()
    messages = []
    if 'messages' in response:
      messages.extend(response['messages'])

    while 'nextPageToken' in response:
      page_token = response['nextPageToken']
      response = service.users().messages().list(userId=user_id, q=query,
                                         pageToken=page_token).execute()
      messages.extend(response['messages'])

    return messages
  except:
    print ('An error occurred: %s' % error)

In [7]:
def FDS(mail_list):

    df = pd.DataFrame(columns = ['id', 'thread_id', 'date', 'from', 'subject'])
    count = 0
    for mail in mail_list:
        m_id = mail['id']
        thread_id = mail['threadId']
        
        message = service.users().messages().get(userId='me', id=m_id).execute()
        payload = message['payload'] 
        header = payload['headers']

        for item in header:
            if item['name'] == 'From':
                sender = item['value']
                print('From: {}'.format(sender))
            if item['name'] == 'Date':
                date = item['value']
                print('Date: {}'.format(date))
            if item['name'] == 'Subject':
                subject = item['value']
                print('Subject: {}'.format(subject))    

        print(' ')
        print(' ')
        
        ## writing to frame
        df.at[count, 'id'] = m_id
        df.at[count, 'thread_id'] = thread_id
        df.at[count, 'date'] = date
        df.at[count, 'from'] = sender
        df.at[count, 'subject'] = subject

        count += 1
        
    
    
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [8]:
## Credentials

cwd = os.getcwd()
path_cred = os.path.join(cwd, 'gmail_credentials')

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            os.path.join(path_cred, 'credentials.json'), SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [9]:
service = build('gmail', 'v1', credentials=creds)

In [10]:
## specifying the path to database:
cwd = os.getcwd()
path_data = os.path.join(cwd, '_data_output')
print(path_data)

/home/leandro/Desktop/github_reps/sigo/_data_output


In [11]:
os.listdir(path_data)

['MainTrix_BoletosAds.csv',
 'oficios_google_clean.csv',
 'MainTrix_Writs.csv',
 'main_trix_clean.csv',
 'controle_linhas_clean.csv',
 'MainTrix_Segmentation.csv']

In [12]:
df_trix = pd.read_csv(os.path.join(path_data, 'main_trix_clean.csv'), sep = '\t', encoding = 'utf-8')

In [13]:
df_trix

,line,Case # (DLBCA and Google reference),UID,Date of the Court Orders,Court,same court,Lawsuit Number,CP?,Summary of the request/what to produce [internal team rec],STQ confirm scope of production,...,Previous Orders and Answers,Google's proposed answer,LIS: \nInformatin about case,LIS: \nPolicy and content check,LIS: \nGoogle Inc answer sent to authority? which one?,LIS:\nPriority,LIS:\nAssignee,Status,Status - writ of mandamus,Law Firm
0,1,\n,NaN,NaN,2018-01-17 00:00:00,Yes,0009460-19.2013.403.6181,No,\n,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,"Andre reviewed and signed, but missing affidav...",Account status: live.,Speech: No CRC (Gmail) - No speech related inv...,May 23rd: Felipe shared content and letters wi...,P0,Completed\n(Felipe),LIS Status: May 22: waiting on Matt/Trev's dec...,A writ was not filed in this case,A writ was not filed in this case
1,2,6-9080000028545,valdeci.silvacampos@gmail.com\nevangelina.pinh...,1st Court Order – 11.23.2012\n2nd Court Order ...,5th Federal Criminal Court of São Paulo,Yes,0002618-91.2011.403.6181,No,Google received a court order to provide conte...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,NaN,Speech: Not enough information. Nature of inve...,*all content preserved is linked to matters re...,Content and letters shared with OC. Download o...,P0 - 04,Completed \n(Keyla 5/20),"50,000 BRL Daily fine and threat of contempt. ...",DLBCA filed a writ of mandamus before TRF-3. I...,DLBCA
2,3,9-5821000001559 FPO Class Action Case,reichswehr88.blogspot.com.br,1st Court Order – 08.07.2013\n2nd Court Order ...,9th Federal Criminal Court of São Paulo,Yes,0007067-24.2013.403.6181,No,Google received court order to provide data/co...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,https://docs.google.com/a/google.com/document/...,LIS preserved content.,Content: preserved for the same matter / Check...,Content and letters shared with OC. Download o...,P0 - 05,Completed \n(Felipe 5/19),"50,000 BRL daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TRF-3. D...,DLBCA
3,4,5-6191000002205,alvarado.alejandro@gmail.com,Court Order - 11.07.2013,8th Criminal Court of Goiânia - Goiás State,No,201003908777,No,Google received court order to provide and pre...,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Content: CO asks for historical content + 15 d...,Content and letters shared with OC. Download o...,P0 - 03,Completed\n(Felipe 5/20),"10,000 BRL Daily fine and no threat of contemp...",DLBCA filed a writ of mandamus before TJGO. In...,DLBCA
4,5,[9-9709000000828] ),quarne.ba@gmail.com\njosemarcos.quarne@gmail.com,1st Court Order – 03.22.2013\n2nd Court Order ...,2nd Criminal Court of Salvador - Bahia State,No,0076690-72.2011.805.0001,No,NaN,NaN,...,https://drive.google.com/a/dlbca.com.br/?tab=m...,Andre reviewed and signed,LIS preserved content.,Speech: Investigation of crimes against the ta...,May 21: Content and letters shared with OC. Do...,P1 - 04,Completed \n(Felipe),"10,000 BRL daily fine and no threat of contemp...",Not filed writ in this case,Not filed writ in this case
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,7740,[7-9812000028504],"josxxx -- Gmail, emixxx -- Gmail, robxxx -- Gm...",Court Order 2019/12/09\n \nReceived: 2019/12/19\n,13ª Vara Federal de Curitiba/PR,NaN,5075998-22.2019.4.04.7000,NaN,NaN,LEGAL OPINION: \n1/ Background: This a Lava-Ja...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7615,7741,[6-8938000029130],"ferxxx -- Gmail, advxxx -- Gmail, CPF’s",Court Order 2019/11/25\n \nReceived: 2019/12/19\n,Vara dos Feitos Relativos a Delitos Praticados...,NaN,0330591-87.2019.8.05.0001,NaN,NaN,LEGAL OPINION: \n1/ Background: n/a\n \n2/ Leg...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7616,7742,[0-5628000028599],"elixxx -- Gmail, rahxxx -- Gmail, rhaxxx --Gma...",Court Order 2019/12/17\n \nReceived: 2019/12/19\n,1ª Vara

In [22]:
## Looking at today:

today_date = str(datetime.today().date())
min_hour = '00:00:00'
max_hour = '23:59:59'

results = DateConv(' '.join([today_date, min_hour]))
start_time = results[0]
print(start_time)

results = DateConv(' '.join([today_date, max_hour]))
end_time = results[0]
print(end_time)


mail_list = ListMessagesMatchingQuery(service, 'me', query='after:{} before:{}'.format(start_time, end_time))

print(len(mail_list))
df_today = FDS(mail_list)

1581649200.0
1581735599.0
1
Date: Fri, 14 Feb 2020 12:44:38 +0000
Subject: FWD: [9-1210000029610] BR Data Request IPL 09/2020
From: lis-latam@google.com
 
 


In [23]:
df_today

,id,thread_id,date,from,subject
0,17043bac9a200af2,17043bac9a200af2,2020-02-14 12:44:38+00:00,lis-latam@google.com,FWD: [9-1210000029610] BR Data Request IPL 09/...


In [24]:
df_group = df_today.groupby('thread_id')['id'].apply(list).to_frame().reset_index()
df_group.columns = ['thread_id', 'id_list']
df_group['mail_counter'] = df_group['id_list'].apply(len)

In [25]:
df_group

,thread_id,id_list,mail_counter
0,17043bac9a200af2,[17043bac9a200af2],1


In [26]:
type(df_today['subject'][0])

str

In [27]:
test = df_today['subject'][0]
FindNumbersBraket(test)


['9-1210000029610']

In [28]:
df_today['subject'].apply(FindNumbersBraket)

0    [9-1210000029610]
Name: subject, dtype: object

In [29]:
# Look for labels:
results = service.users().labels().list(userId='me').execute()
labels = results.get('labels', [])

if not labels:
    print('No labels found.')
else:
    print('Labels:')
    for label in labels:
        print(label['name'])

Labels:
CATEGORY_PERSONAL
CATEGORY_SOCIAL
IMPORTANT
CATEGORY_UPDATES
CATEGORY_FORUMS
CHAT
SENT
INBOX
TRASH
CATEGORY_PROMOTIONS
DRAFT
SPAM
STARRED
UNREAD


In [30]:
## Query a search term:
mail_list = ListMessagesMatchingQuery(service, 'me', query='0-1811000027487')



In [31]:
mail_list

[{'id': '17036a62b7779e3c', 'threadId': '17036a62b7779e3c'},
 {'id': '16dcc1f0f9c717d5', 'threadId': '16d2c847a626d4f0'},
 {'id': '16d2c847a626d4f0', 'threadId': '16d2c847a626d4f0'},
 {'id': '16cdf8d761dc59a7', 'threadId': '16cdf8d761dc59a7'},
 {'id': '16cdaec76bf5cc8b', 'threadId': '16c638ea0cd8828d'},
 {'id': '16c638ea0cd8828d', 'threadId': '16c638ea0cd8828d'},
 {'id': '16c541df77a62fd1', 'threadId': '16c541df77a62fd1'}]

In [32]:
df_fds = FDS(mail_list[:10])

From: Mohamad Hassan Fares <mohamad.fares@carinaquito.com>
Date: Tue, 11 Feb 2020 20:46:42 -0300
Subject: [1-9786000029412] - FEDERAL - JFRJ - c 2020/02/21 - February 21st, 2020 - CRIMINAL - GMAIL - BID RIGGING - CRIMINAL ASSOCIATION - TRIX #8057 - 5th Request
 
 
Date: Mon, 14 Oct 2019 21:14:10 +0000
Subject: Re: [4-1419000028000] - FEDERAL - JFRJ - c 2019/09/23 - September 23rd, 2019 - CRIMINAL - GMAIL - BID RIGGING - CRIMINAL ASSOCIATION - TRIX #6801 - 3rd Request
From: lis-latam@google.com
 
 
From: Mohamad Fares <mohamad.fares@carinaquito.com>
Subject: [4-1419000028000] - FEDERAL - JFRJ - c 2019/09/23 - September 23rd, 2019 - CRIMINAL - GMAIL - BID RIGGING - CRIMINAL ASSOCIATION - TRIX #6801 - 3rd Request
Date: Fri, 13 Sep 2019 18:25:36 -0300
 
 
From: Helena Rossi <helena.rossi@carinaquito.com>
Subject: PRODUÇÃO >  [0-1811000027487] - c 2019/08/14 - August 14th, 2019 - FEDERAL - JFRJ - CRIMINAL - GMAIL - BID RIGGING - CRIMINAL ASSOCIATION - TRIX #6382 - 2nd Request
Date: Thu, 29 

In [33]:
df_fds

,id,thread_id,date,from,subject
0,17036a62b7779e3c,17036a62b7779e3c,2020-02-11 20:46:42-03:00,Mohamad Hassan Fares <mohamad.fares@carinaquit...,[1-9786000029412] - FEDERAL - JFRJ - c 2020/02...
1,16dcc1f0f9c717d5,16d2c847a626d4f0,2019-10-14 21:14:10+00:00,lis-latam@google.com,Re: [4-1419000028000] - FEDERAL - JFRJ - c 201...
2,16d2c847a626d4f0,16d2c847a626d4f0,2019-09-13 18:25:36-03:00,Mohamad Fares <mohamad.fares@carinaquito.com>,[4-1419000028000] - FEDERAL - JFRJ - c 2019/09...
3,16cdf8d761dc59a7,16cdf8d761dc59a7,2019-08-29 19:44:43-03:00,Helena Rossi <helena.rossi@carinaquito.com>,PRODUÇÃO > [0-1811000027487] - c 2019/08/14 -...
4,16cdaec76bf5cc8b,16c638ea0cd8828d,2019-08-29 01:10:25+00:00,lis-latam@google.com,RE: [0-1811000027487] - c 2019/08/14 - August ...
5,16c638ea0cd8828d,16c638ea0cd8828d,2019-08-05 17:52:36-03:00,Helena Rossi <helena.rossi@carinaquito.com>,[0-1811000027487] - c 2019/08/14 - August 14th...
6,16c541df77a62fd1,16c541df77a62fd1,2019-08-02 20:55:43+00:00,lis-latam@google.com,FWD: [0-1811000027487] EXPEDIENTE SIGILOSO


In [34]:
## download attachments:
#for item in mail_list[:10]:
#    mid = item['id']
#    GetAttachments(service, 'me', mid)

In [35]:
## Get message:
test = GetMessage(service, 'me', '16dcc1f0f9c717d5')

Message snippet: Dear all, I have just shared Drive folder 2756332(#6801) with quito-share@googlegroups.com and Taís containing production and letters. Thanks, Fernando E. and Alexa- LIS On Fri, Sep 13, 2019 at 21:25


In [36]:
## Searching a date range

results = DateConv('2020-02-10 00:00:00')
start_time = results[0]
print(start_time)

results = DateConv('2020-02-13 18:00:00')
end_time = results[0]
print(end_time)


mail_list = ListMessagesMatchingQuery(service, 'me', query='after:{} before:{}'.format(start_time, end_time))

1581303600.0
1581627600.0


In [37]:
mail_list

[{'id': '17040422301277a2', 'threadId': '17040422301277a2'},
 {'id': '1704014b523ad95a', 'threadId': '17036b491219fd77'},
 {'id': '170400de25cd7dac', 'threadId': '1703bab743b6f57b'},
 {'id': '1703ffe43d21d0a5', 'threadId': '1703ffe43d21d0a5'},
 {'id': '1703ff6ef1192a2e', 'threadId': '1703ff6ef1192a2e'},
 {'id': '1703fefd661d5a8f', 'threadId': '1703fefd661d5a8f'},
 {'id': '1703fed40ed29df2', 'threadId': '17036a50e08bcfcb'},
 {'id': '1703faa326c650c2', 'threadId': '1703faa326c650c2'},
 {'id': '1703eff15323ca31', 'threadId': '17021c0c7c1053f6'},
 {'id': '1703ed98a55f752d', 'threadId': '17031b633ba5ef14'},
 {'id': '1703ec09eaab2878', 'threadId': '1703ec09eaab2878'},
 {'id': '1703bbe141599d65', 'threadId': '1703ba85c84a0981'},
 {'id': '1703bab743b6f57b', 'threadId': '1703bab743b6f57b'},
 {'id': '1703baa78a44ecb0', 'threadId': '1703baa78a44ecb0'},
 {'id': '1703ba85c84a0981', 'threadId': '1703ba85c84a0981'},
 {'id': '1703ba7025a99474', 'threadId': '17034fbd4bc7926d'},
 {'id': '1703ba37f3c860c

In [38]:
df_fds = FDS(mail_list[:10])

Date: Thu, 13 Feb 2020 20:34:00 +0000
Subject: FWD: [0-8003000029452] Autorização
From: lis-latam@google.com
 
 
Date: Thu, 13 Feb 2020 19:44:23 +0000
Subject: Re: [4-4364000029622] - FINES - URGENT 72 HOURS - c 2020/02/13 - February 13th, 2020 - CRIMINAL - GMAIL - MURDER - TRIX #8059
From: lis-latam@google.com
 
 
Date: Thu, 13 Feb 2020 19:36:55 +0000
Subject: Re: [0-1638000029307] - (IMEI 2nd phase) - 2020/02/17 - February 17th, 2020 - CRIMINAL - GMAIL - CRIMINAL ORGANIZATION - DRUG TRAFFICKING - ARM TRAFFICKING - TRIX #8019 - 6th Request
From: lis-latam@google.com
 
 
Date: Thu, 13 Feb 2020 19:19:52 +0000
Subject: FWD: [8-7373000029650] 132a DP - Arraial do Cabo - Polícia Civil do Estado do Rio de Janeiro/BR - Número de referência  3102154
From: lis-latam@google.com
 
 
Date: Thu, 13 Feb 2020 19:11:50 +0000
Subject: FWD: [6-5229000029564] BR Data Request 072/2020-C–GAB/VDT/P F/RO
From: lis-latam@google.com
 
 
Date: Thu, 13 Feb 2020 19:04:05 +0000
Subject: FWD: [1-5710000029265] BR 